**Data Scraping**

This program is designed to fetch details and comments from a specific YouTube video using the YouTube Data API. It retrieves video metadata, including the title, channel details, view count, and upload date, as well as all comments and their replies. The comments are saved into a CSV file named after the video title.


**Usage:**

1. Obtain a YouTube Data API Key:

*   Go to the Google Developers Console.
*   Create a new project or select an existing one.
*   Enable the YouTube Data API v3.
*   Create an API key and copy it.

2. Set the API Key:

*   Replace the DEVELOPER_KEY value in the code with your API key.

3. Run the Program:

*   Ensure you have the correct video ID for the YouTube video you want to analyze.
*   Replace the video_id variable value with your desired video ID.
*   Run the program
*   The program will fetch video details and comments, and save the comments to a CSV file named after the video title.


**Notes:**

*   The YouTube Data API has usage quotas and rate limits. Ensure your usage complies with these limits to avoid service interruptions.

*   The program currently fetches comments in batches of 100. Modify the maxResults parameter if you need to adjust the number of comments fetched per request.


**Potential Errors and Fixes:**

1. API Key Errors - If you encounter an error related to the API key, ensure that it is correctly copied and that the YouTube Data API v3 is enabled in your Google Developers Console.

2. Data Handling - If there are issues with saving the CSV file, ensure the video title does not contain special characters that are not allowed in file names.

3. Rate Limits - If you exceed the API rate limit, you might need to wait before making additional requests or consider optimizing the number of requests made.

In [ ]:
 # Data Scraping Program for Youtube Comments Using API Key
 # Only does 1 video

# Import necessary modules for interacting with YouTube API and handling data
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

# Define the API service name and version
api_service_name = "youtube"
api_version = "v3"
# Define your YouTube API developer key
DEVELOPER_KEY = "INSERT YOUR API KEY HERE"

# Build the YouTube API client using the specified service name, version, and developer key
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

# Function to fetch details of a given video using its video ID
def fetch_video_details(video_id):
    # Request video details (snippet and statistics) for the specified video ID
    video_request = youtube.videos().list(
        part="snippet,statistics",
        id=video_id
    )
    video_response = video_request.execute()
    video_details = video_response['items'][0]

    # Extract video details: title, channel ID, view count, and upload date
    video_title = video_details['snippet']['title']
    channel_id = video_details['snippet']['channelId']
    view_count = video_details['statistics']['viewCount']
    upload_date = video_details['snippet']['publishedAt']

    # Request channel details using the channel ID extracted from the video details
    channel_request = youtube.channels().list(
        part="snippet,statistics",
        id=channel_id
    )
    channel_response = channel_request.execute()
    channel_details = channel_response['items'][0]

    # Extract channel details: title and subscriber count
    channel_title = channel_details['snippet']['title']
    subscriber_count = channel_details['statistics']['subscriberCount']

    # Return the fetched video and channel details
    return video_title, channel_title, subscriber_count, view_count, upload_date

# Function to fetch comments for a given video using its video ID
def fetch_comments(video_id):
    comments = []  # Initialize an empty list to store comments

    # Request the first set of comments for the specified video ID
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100  # Maximum number of comments to fetch per request
    )
    response = request.execute()

    # Loop through the response and collect comments
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']

            # Append the comment details to the comments list
            comments.append({
                'Author': comment['authorDisplayName'],
                'Comment Text': comment['textDisplay'],
                'Date Published': comment['publishedAt'],
                '# of Likes': comment['likeCount'],
                'Reply Count': item['snippet']['totalReplyCount']
            })

            # Check if the comment has replies and fetch them if there are any
            if item['snippet']['totalReplyCount'] > 0:
                parent_id = item['snippet']['topLevelComment']['id']
                replies_request = youtube.comments().list(
                    part="snippet",
                    parentId=parent_id,
                    maxResults=100
                )
                replies_response = replies_request.execute()

                for reply in replies_response['items']:
                    reply_snippet = reply['snippet']

                    # Append the reply details to the comments list
                    comments.append({
                        'Author': reply_snippet['authorDisplayName'],
                        'Comment Text': reply_snippet['textDisplay'],
                        'Date Published': reply_snippet['publishedAt'],
                        '# of Likes': reply_snippet['likeCount'],
                        'Reply Count': ''  # Reply count is not applicable for replies
                    })

        # Check if there are more pages of comments to fetch
        if 'nextPageToken' in response:
            # If there are more pages, fetch the next set of comments
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=100,
                pageToken=response['nextPageToken']
            )
            response = request.execute()
        else:
            # If there are no more pages, exit the loop
            response = None

    # Return the list of collected comments
    return comments

# Define the video ID of the video you want to fetch details and comments for
video_id = "INSERT VIDEO ID HERE"  # Replace with the desired video ID

# Fetch video details and comments for the specified video ID
video_title, channel_title, subscriber_count, view_count, upload_date = fetch_video_details(video_id)
comments = fetch_comments(video_id)

# Create a DataFrame from the collected comments
df = pd.DataFrame(comments)

# Save the DataFrame to a CSV file with the video title as the filename (spaces replaced with underscores)
csv_filename = f"{video_title.replace(' ', '_')}.csv"
df.to_csv(csv_filename, index=False)

# Print video details and the filename of the saved CSV
print(f"Video Title: {video_title}")
print(f"Channel Title: {channel_title}")
print(f"Subscriber Count: {subscriber_count}")
print(f"View Count: {view_count}")
print(f"Upload Date: {upload_date}")
print(f"Comments have been saved to {csv_filename}")